In [21]:
import os
import astropy
import numpy as np
from astropy.table import Table
from astropy.table import Column
import glob
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from mpl_toolkits.mplot3d import Axes3D

In [22]:
sample = Table.read("/home/xhall/Documents/NewZTF/ML_sample.ascii", format = "ascii")
sample.rename_column('col1', 'ZTF_Name')
sample.rename_column('col2', "Class")
sample.rename_column('col8', "Version")

In [23]:
np.size(np.unique(sample["ZTF_Name"]))

4411

In [24]:
snidoutput = np.load("/home/xhall/Documents/NewZTF/SNIDoutput100.npy",allow_pickle=True)

In [25]:
snidoutput[0][0]

'AT2018hrg_20181103_P60_v1'

In [26]:
ResultsTable_SNID = Table(
                    names=("Version", "c_rlap", "c_snid"
                    ),
                    meta={"name": "Class Results"},
                    dtype=("U64", "float64", "U64"
                          )
                    )
for j in snidoutput:
    row = []
    row.append(j[0] + ".ascii")
    row.append(j[1][0]["rlap"])
    row.append(j[1][0]["type"])
    ResultsTable_SNID.add_row(row)

In [27]:
ZTable_best = Table(
                    names=("Version", "sntemplate", "z_rlap", "z_snid", "z_snid_err", "age", "age_flag", "z_level"
                    ),
                    meta={"name": "Redscore Results"},
                    dtype=("U64", "U64", "float64", "float64", "float64","float64", "float64", "int32"
                          )
                    )
for j in snidoutput:
    row = []
    row.append(j[0] + ".ascii")
    row.append(j[1][0]["sn"])
    good = j[1][np.where(j[1]["grade"] == "good")]
    good = good[np.where(good["z"] <= .2)]
    if(len(good) != 0):
        row.append(good[0]["rlap"])
        row.append(float(good[0]["z"]))
        row.append(float(good[0]["zerr"]))
        row.append(float(good[0]["age"]))
        row.append(float(good[0]["age_flag"]))
        row.append(1)
    else:
        row.append(j[1][0]["rlap"])
        row.append(float(j[1][0]["z"]))
        row.append(float(j[1][0]["zerr"]))
        row.append(float(j[1][0]["age"]))
        row.append(float(j[1][0]["age_flag"]))
        row.append(0)
    ZTable_best.add_row(row)

In [28]:
RedshiftClass = astropy.table.join(astropy.table.join(ResultsTable_SNID, ZTable_best),sample)

In [29]:
RedshiftClass[RedshiftClass["c_snid"] == "Ia-norm"]

Version,c_rlap,c_snid,sntemplate,z_rlap,z_snid,z_snid_err,age,age_flag,z_level,ZTF_Name,Class,col3,col4,col5,col6,col7,col9
str64,float64,str64,str64,float64,float64,float64,float64,float64,int32,str12,str12,float64,float64,float64,float64,float64,int64
ZTF17aaazdba_20190309_Gemini_N_v2.ascii,4.77,Ia-norm,sn2007ci,4.77,0.1967,0.0111,18.4,0.0,1,ZTF17aaazdba,TDE,0.022,nan,2458551.5,2458561.68,-10.18,0
ZTF17aaazdba_20190310_DCT_v1.ascii,7.86,Ia-norm,sn1999cc,7.86,0.0223,0.0075,18.2,0.0,1,ZTF17aaazdba,TDE,0.022,nan,2458552.5,2458561.68,-9.18,0
ZTF17aaazdba_20190316_NOT_v1.ascii,6.03,Ia-norm,sn2002do,6.03,0.0332,0.0094,22.6,0.0,1,ZTF17aaazdba,TDE,0.022,nan,2458558.5,2458561.68,-3.18,0
ZTF17aaazdba_20190424_P60_v1.ascii,7.32,Ia-norm,sn90N,7.32,0.9621,0.0085,-6.4,0.0,0,ZTF17aaazdba,TDE,0.022,nan,2458597.5,2458561.68,35.82,0
ZTF17aaazdba_20190501_Keck1_v1.ascii,7.64,Ia-norm,sn2001G,7.33,0.0341,0.0113,32.7,0.0,1,ZTF17aaazdba,TDE,0.022,nan,2458604.5,2458561.68,42.82,0
ZTF17aaazdba_20190505_Lick_3-m_v1.ascii,9.38,Ia-norm,sn95E,9.38,0.0358,0.009,32.7,0.0,1,ZTF17aaazdba,TDE,0.022,nan,2458608.5,2458561.68,46.82,0
ZTF17aaazdba_20190529_DCT_v1.ascii,6.84,Ia-norm,sn1996ai,6.84,0.0287,0.0094,25.0,0.0,1,ZTF17aaazdba,TDE,0.022,nan,2458632.5,2458561.68,70.82,0
ZTF18aaaqexr_20200215_P60_v1.ascii,10.11,Ia-norm,sn1991M,10.11,0.7297,0.0057,111.8,0.0,0,ZTF18aaaqexr,Ia,0.0702,nan,2458894.5,2458891.87,2.63,0
ZTF18aabcvnq_20180604_P60_v1.ascii,6.96,Ia-norm,sn2005cf,6.96,0.1741,0.0084,-2.4,0.0,1,ZTF18aabcvnq,star,nan,nan,2458273.5,2458260.62,12.88,0


In [30]:
RedshiftClass.to_pandas().to_csv("/home/xhall/Documents/NewZTF/ML_sample_snid100.csv", index=False)

In [11]:
RedshiftClass = Table.from_pandas(pd.read_csv("/home/xhall/Documents/NewZTF/ML_sample_snid.csv"))

In [12]:
len(np.unique(RedshiftClass[np.where(RedshiftClass["z_snid"] < .2)]["ZTF_Name"]))

3860

In [13]:
len(np.unique(RedshiftClass["ZTF_Name"]))

4323